# Time Series Analysis - ARIMA, SARIMA, Prophet, LSTM.

Author: *Sergio Alberto Mora Pardo*

* email: [sergiomora823@gmail.com](mailto:sergiomora823@gmail.com)
* LinkedIn: [/sergiomorapado](https://www.linkedin.com/in/sergiomorapardo/)
* GitHub: [sergiomora03](https://github.com/sergiomora03)

En el siguiente notebook se analiza la serie de tiempo de producción de cerveza de Australia. Se aplican los modelos ARIMA, SARIMA. También, se prueba el ajuste con Prophet y finalmente, se crea una Red Neuronal Recurrente.


1. Choosing your time series (https://www.kaggle.com/shenba/time-series-datasets).

2. Analysis of the context of the problem and relevance of the analysis: This should answer the question of why it is interesting or important to study the selected data.

3. Understanding the data: Understand and analyze the main components of the time series, for example: seasonality, cyclicity, autocorrelation, behavior of the residuals, among others. Make use of graphs that allow you to understand each of the components clearly.

4. Application of the model in predictions: Correct use of the models seen in class to make predictions related to the problem of interest.

5. Conclusions: The conclusions must be relevant to the problem of interest. Conclusions on the procedures performed are also expected.

# 1. Serie seleccionada

La serie seleccionada para la realización del presente proyecto corresponde a la producción mensual de cerveza en Australia en unidades de volumen para el periodo: enero de 1956 - agosto de 1995 (un total de 476 observaciones).

# 2. Análisis y relevancia de la serie


**Antecedentes:**

El sector Cervecero en Australia hace parte de la actividad económica del Comercio, el cual aporta aproximadamente un 7% de la actividad económica del país. Dentro del comercio, el Retail lidera el sector con un aporte del 52% donde se ubica el sector Cervecero. 

La Industria Cevecera Australiana inicialmente fue dominada por los productores nacionales tradicionales Carlton & United Breweries (CUB) y León  principalmente. A finales de la década de los 70, empezaron una serie de fusiones de estas empresas regionales con trasnacionales, transformando el mercado a una gran participación extranjera. Actualmente, el mercado lo dominan principalmente: La Cervera CUB que en 2011 se fusionó con la multinacional Sabmiller y hoy en día es 100% propiedad de AB InBev y,  por otra parte, la cervecera León en 2009 pasó a propiedad de la Cervecera Japonesa  Kirin Holdings Company Limited. Estas dos competidores hacen aproximadamente el 89% del mercado. El resto del mercado lo ocupa en mayor medida la Cervecera Nacional Coopers con un 4% de participación.  

**Relevancia e Accionabilidad:** 

El análisis del mercado de Cervezas y específicamente el de la producción es de vital importancia para las industrias cerveceras del País. Es necesario hacer el tracking mensual del mercado y así mismo prever la producción de Cerveza que cubra toda la demanda del mercado. El análisis de esta serie de tiempo de  la producción de Cerveza, le permite a los grandes fabricantes planear, organizar todo el proceso productivo de la manera más óptima posible,  por medio de un anticipo o pronóstico del comportamiento del mercado. Como veremos más adelante en esta serie existen principalmente dos grandes picos de estacionalidad, marcados en primera medida por la temporada navideña y verano (Dic-Ene) y en segunda medida por las vacaciones de mitad de año(Jun-Jul), en donde se observa un incremento muy marcado en la demanda y consumo de Cerveza y por lo tanto tambien en la producción. Por otra parte, les ayuda a entender a los fabricantes cual es el mejor momento o la fecha ideal para realizar lanzamientos de marcas/submarcas en el mercado o para realizar alguna campaña específica en algún sector, de acuerdo a las capacidades de producción que tengan previstas.

**Escalabilidad del Modelo:** 

Actualmente, el mercado al ser dominado principalmente por inversión extranjera, a las multinacionales no les interesa únicamente hacer tracking sobre la producción de Australia, sino también aplicar estos modelos en los demás países  donde tienen participación. La ventaja del sector cervecero es que en muchos países el comportamiento estacional de Diciembre y Junio se mantiene tanto en ventas como en producción, lo que hace que estos modelos y análisis de mercado tengan una fácil escalabilidad en datos de la industria de otros países, además que los modelos y algoritmos aquí utilizados permiten una facil adaptación.





# 3. Entendiendo los datos

### 3.1 instalando librerias

In [ ]:
#!pip uninstall statsmodels

In [ ]:
 #!pip install statsmodels --upgraded

In [ ]:
#!pip install pyramid-arima

In [ ]:
!pip install pmdarima # libreria con la función auto_arima()

### 3.2 Importando Librerias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from scipy import stats
from matplotlib import pyplot
from pmdarima import auto_arima
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from fbprophet import Prophet


%matplotlib inline
sns.set()

### 3.3 Importando datos

In [ ]:
url = 'https://raw.githubusercontent.com/sergiomora03/AdvancedMethodsDataAnalysis/master/datasets_56102_107707_monthly-beer-production-in-austr.csv'
data = pd.read_csv(url)
data.head()

In [ ]:
#Descripción de la base de datos
data.info()

De acuerdo a lo observado en la data original es necesario transformar la variable **Month** a formato fecha

In [ ]:
data.Month = pd.to_datetime(data.Month)
data.set_index('Month', inplace=True)
data.head()

### 3.4. Transformación y descompocisión

In [ ]:
#Producción en logaritmo
data['Log_Production']=np.log(data['Monthly beer production'])

In [ ]:
#Evolución de producción de cerveza en Australia durante el periodo de análisis (niveles y logaritmo)
data[['Monthly beer production']].plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=15);
data[['Log_Production']].plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=15);

Los anteriores gráficos permiten observar la evolución de la producción de cerveza en Australia para el periodo de análisis.

In [ ]:
res = seasonal_decompose(data['Monthly beer production'], model='additive',freq=12)

def plotseasonal(res, axes ):
    res.observed.plot(ax=axes[0], legend=False)
    axes[0].set_ylabel('Observed')
    res.trend.plot(ax=axes[1], legend=False)
    axes[1].set_ylabel('Trend')
    res.seasonal.plot(ax=axes[2], legend=False)
    axes[2].set_ylabel('Seasonal')
    res.resid.plot(ax=axes[3], legend=False)
    axes[3].set_ylabel('Residual')

fig, axes = plt.subplots(ncols=1, nrows=4, sharex=True, figsize=(20,5))

plotseasonal(res, axes)

plt.tight_layout()
plt.show()


In [ ]:
res = seasonal_decompose(data['Log_Production'], model='additive',freq=12)

fig, axes = plt.subplots(ncols=1, nrows=4, sharex=True, figsize=(20,5))

plotseasonal(res, axes)

plt.tight_layout()
plt.show()

A partir de los gráficos de la producción de cerveza (tanto en niveles como en logaritmo), se puede observar lo siguiente en relación con el comportamiento de la serie y sus componentes:

1) Hasta el año 1974 se registra un crecimiento importante de la producción de cerveza en el país, mostrando un tendencia positiva durante estos primeros años de análisis, para luego estabilizarse y tener una ligera disminución a principios de la decada de los 80´s (tendencia negativa). La presencia de diferentes tendencias en el tiempo, indican inicialmente que la serie no es estacionaria.

2) La serie tiene un marcado componente estacional, que consiste en un aumento de la producción de cerveza cada 12 meses, especificamente a finales del año (noviembre - diciembre) para luego disminuir a partir del mes de enero del siguiente año. Esta dinámica es consistente con la temporada de verano de Australia.

3) El cambio en la amplitud del cada ciclo y la presencia de un componente aleatorio en el periodo de análisis, indican que la producción de cerveza ha recibido diferentes choques (oferta o demanda) que ha cambiado su dinámica en el tiempo y, además, puede indicar problemas de heterocedasticidad en la serie.

### 3.5. Análisis de estacionariedad

In [ ]:
beer=data['Monthly beer production']
Log_beer=data['Log_Production']

In [ ]:
#Primera diferencia de la serie (niveles)
beer.diff().plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=10);

In [ ]:
#Primera diferencia de la serie (logaritmo)
Log_beer.diff().plot(figsize=(20,5), linewidth=2, fontsize=10)

Una vez se diferencian las series se observa que la tendencia desaparece. Adicionalmente, con la transformación de logaritmo parece mejorar el problema de varianza no constante. Por consiguiente, el ejercicio de análisis y estimación se realizará con **la serie en logaritmo**.

No obstante, a continuación, se procede a realizar la prueba de Dickey- Fuller para realizar una verificación formal de la estacionariedad de la serie.

In [ ]:
#Prueba Dickey- Fuller
result = adfuller(data['Log_Production'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

Los resultados de la prueba de Dickey- Fuller confirman que, con un nivel de significancia del 5%, la serie de producción de cerveza no es estacionaria.

In [ ]:
#Prueba Dickey- Fuller - primera diferencia
result = adfuller(data['Log_Production'].diff().iloc[1:])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

Los resultados de la prueba de Dickey- Fuller confirman que, con un nivel de significancia del 5%, la serie de producción de cerveza diferenciada si es estacionaria.

In [ ]:
#Autocorrelación de la serie en análisis
plt.figure(figsize=(20,5))
pd.plotting.autocorrelation_plot(data['Log_Production']);

Como se comprobó en el test de raíz unitaria, esta caída lenta en la función de autocorrelación simple, da indicios que la serie es no estacionaria.

In [ ]:
#Autocorrelación de la primera diferencia de la serie en análisis
plt.figure(figsize=(20,5))
pd.plotting.autocorrelation_plot(data['Log_Production'].diff().iloc[1:]);

In [ ]:
#Autocorrelación simple y parcial serie producción cerveza
fig, ax = plt.subplots(1,2,figsize=(20,5))
plot_acf(data['Log_Production'], ax=ax[0])
plot_pacf(data['Log_Production'], ax=ax[1])
plt.show()

In [ ]:
#Autocorrelación simple y parcial primera diferencia de serie producción cerveza
fig, ax = plt.subplots(1,2,figsize=(20,5))
plot_acf(data['Log_Production'].diff().iloc[1:], ax=ax[0])
plot_pacf(data['Log_Production'].diff().iloc[1:], ax=ax[1])
plt.show()

# 4. Aplicación de modelos de predicción

En la presente sección se procede a la estimación de diferentes modelos de predicción: ARIMA, SARIMA, Prophet y una red reuronal recurente con una celda LSTM.

## 4.1. Modelo ARIMA

#### 4.1.1 Selección proceso ARIMA a estimar

In [ ]:
X = data['Log_Production'].values
size = int(len(X) * 0.9)
train, test = X[0:size], X[size:len(X)]


def ARIMA_FUNCTION(p,q):
	for t in range(len(test)):
		model_1 = ARIMA(history, order=(p,1,q))
		model_fit_1 = model_1.fit(disp=0)
		output = model_fit_1.forecast()
		yhat = output[0]
		predictions.append(yhat)
		obs = test[t]
		history.append(obs)
	return mean_squared_error(test, predictions)**0.5

In [ ]:
results=[]
for i in range(3):
    for j in range(4):
        if (i>j or i==0) and (i-j<3) :
            predictions = list()
            history = [x for x in train]
            results.append([i,j,ARIMA_FUNCTION(i,j)])
            print((i,j))

Escenarios para evaluar los procesos *ARIMA*, en estos casos en todos los modelos utilizamos 1 Diferenciación, el primer parámetro representa el proceso AR, el segundo corresponde al MA

In [ ]:
results

De acuerdo a los RMSE obtenidos para cada modelo evaluado en los procesos ARIMA, el mejor modelo es el ARIMA (0,1,2) con un RMSE correspondiente a 0.12223215571124342

#### 4.1.2 Estimación modelo

In [ ]:
model = ARIMA(train, order=(0,1,2))
model_fit = model.fit(disp=0)
print(model_fit.summary())

#### 4.1.3. Análisis de residuales

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
residuals.plot(figsize=(20,5))
plt.show()

In [ ]:
residuals.plot(kind='kde', figsize=(20,5))
plt.show()
print(residuals.describe())

De acuerdo a los gráficos de los residuales podemos observar que el proceso tiene media constante, centrada alrededor de cero y varianza constante. Tambien, graficamente parece ajustarse a una distribución Normal, sin embargo validamos esto con una prueba de normalidad 

In [ ]:
print("KS P-value = "+str(round(stats.kstest(residuals, 'norm')[1], 10)))

Según los resultados del test de Kolmogorov Smirnoff con un nivel de significancia del 5% no podemos concluir que los residuales se ajustan a una distribución normal.

#### 4.1.4. Rolling Forecast

In [ ]:
history = [x for x in train]
predictions = list()

for t in range(len(test)):
    model_1 = ARIMA(history, order=(0,1,2))
    model_fit_1 = model_1.fit(disp=0)
    output = model_fit_1.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

In [ ]:
error_ARIMA = mean_squared_error(test, predictions)**0.5
print('Test RMSE: %.3f' % error_ARIMA)

Después de ejecutar el procedimiento de Rolling Forecast con el test el RMSE obtenido es de 0.122 con el ARIMA(0,1,2).

In [ ]:
RollBack=pd.concat([pd.DataFrame({'TEST':test}),pd.DataFrame({'ARIMA':np.concatenate(predictions, axis=0)})],axis=1)
RollBack.head()

#### 4.1.5. Comparación preliminar estimaciones



In [ ]:
RollBack.plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=15);

## 4.2. Modelo SARIMA

Según el análisis de la serie existe un fuerte componente estacional en la producción de cerveza que, como se observó anteriormente, no está siendo capturado por el proceso ARIMA seleccionado. Por consiguiente, se procede a estimar un modelo SARIMA, que es un modelo ARIMA que incluye el componente estacional de las series.

#### 4.2.1. Selección proceso SARIMA a estimar

In [ ]:
stepwise_model = auto_arima(train, 
                            start_p=0,
                            start_q=0, 
                            max_p=5, 
                            max_d=2, 
                            max_q=5, 
                            start_P=1,
                            start_Q=1, 
                            max_P=2, 
                            max_D=2, 
                            max_Q=2, 
                            max_order=10,
                            m=12,
                            seasonal=True,
                            trace=True,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)
print(stepwise_model.aic())

Una vez evaluadas las posibles combinaciones para el proceso SARIMA, el modelo con el mejor performance es el que tiene menor AIC, en este caso es el ARIMA(1,1,1)x(2,0,2,12) con un AIC=-1014.517

#### 4.2.2. Estimación modelo

In [ ]:
mod = sm.tsa.statespace.sarimax.SARIMAX(train, trend='n', order=(1,1,1), seasonal_order=(2,0,2,12))
results = mod.fit()
results.summary()

Evaluando el modelo con los parámetros óptimos, se observa que todos los coeficientes del modelo son significativos a excepción de los primeros coeficientes de AR y MA de la parte estacional, sin embargo los mantenemos en el modelo ya que los segundos coeficientes de AR y MA del componente estacional si son significativos.

#### 4.2.3. Análisis de residuales

In [ ]:
#results.resid
residuals1 = pd.DataFrame(results.resid[1:])
residuals1.plot(figsize=(20,5))
plt.show()

In [ ]:
residuals1.plot(kind='kde', figsize=(20,5))
plt.show()
print(residuals1.describe())

In [ ]:
print("KS P-value = "+str(round(stats.kstest(residuals1, 'norm')[1], 10)))
print("D’Agostino and Pearson’s P-value = "+str(round(stats.normaltest(residuals1, axis=0)[1][0], 6)))

Según los resultados del test de Kolmogorov Smirnoff con un nivel de significancia del 5% no podemos concluir que los residuales se ajustan a una distribución normal.

#### 4.2.4. Rolling Forecast

In [ ]:
X = data['Log_Production'].values
size = int(len(X) * 0.9)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
	model = sm.tsa.statespace.sarimax.SARIMAX(history, trend='n', order=(1,1,1), seasonal_order=(2,0,2,12))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)


In [ ]:
error_SARIMA = mean_squared_error(test, predictions)**0.5
print('Test RMSE: %.3f' % error_SARIMA)

El mejor modelo SARIMA de los testeados presenta un RMSE sobre los datos de test de 0.062.

In [ ]:
RollBack=pd.concat([RollBack,pd.DataFrame({'SARIMA':predictions})],axis=1)
RollBack.head()

In [ ]:
RollBack.plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=15);

## 4.3. Prophet Forecast

#### 4.3.1. Estimación modelo

In [ ]:
data_pf = pd.DataFrame({'ds': data.Log_Production.index[:], 'y': data.Log_Production})
data_pf.head()

In [ ]:
X = data_pf.y
Y = data_pf.ds
size = int(len(X) * 0.9)
train_X, test_X = X[0:size], X[size:len(X)]
train_Y, test_Y = Y[0:size], Y[size:len(Y)]
    
Train = pd.concat([train_Y,train_X], axis=1)
Test = pd.concat([test_Y,test_X], axis=1)

In [ ]:
predictions = list()
    
def rolling_forecast():   
    history = Train.copy()
    
    for t in range(len(test_X)):
        m = Prophet()
        m.fit(history);
        future = m.make_future_dataframe(periods=1, freq='MS')
        forecast = m.predict(future)
        output=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        yhat = output[['yhat']][len(history):].values[0][0]
        predictions.append(yhat)
        obs = pd.DataFrame(Test[['ds','y']].iloc[t])
        history = pd.concat([history, obs.transpose()],axis=0)
        print('predicted=%f, expected=%f' % (yhat, obs.transpose()['y']))

    
    error_PROPHET = mean_squared_error(test_X, predictions)**0.5
    print('Test RMSE: %.3f' % error_PROPHET)

In [ ]:
rolling_forecast()

In [ ]:
error_PROPHET = mean_squared_error(test_X, predictions) **0.5
print('Test RMSE: %.3f' % error_PROPHET)

Para el proceso del Rolling Forecast con Prophet separamos la base en train (90% de obs) para estimar el modelo y en test (10%) para calcular el error de la predicción. Dicho esto, escogemos el modelo con menor RMSE el cual es el que predice mejor, en este caso con un RMSE=0.078

In [ ]:
RollBack=pd.concat([RollBack,pd.DataFrame({'Prophet':predictions})],axis=1)
RollBack.head()

In [ ]:
RollBack[['TEST', 'Prophet']].plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=15);

In [ ]:
RollBack.plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=15);

## 4.4. LSTM Forecast

In [ ]:
data_LSTM = pd.DataFrame({'Log_Production': data.Log_Production})
data_LSTM.head()

In [ ]:
Y = data_LSTM
size = int(len(Y) * 0.9)

train_Y, test_Y = Y[0:size], Y[size:len(Y)]

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_Y)
scaled_train_data = scaler.transform(train_Y)
scaled_test_data = scaler.transform(test_Y)

In [ ]:
n_input = 12
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(200, input_shape=(n_input, n_features)))
#lstm_model.add(LSTM(units=50, return_sequences = True))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

lstm_model.summary()

In [ ]:
lstm_model.fit_generator(generator,epochs=20)

In [ ]:
losses_lstm = lstm_model.history.history['loss']
plt.figure(figsize=(20,5))
plt.xticks(np.arange(0,21,1))
plt.plot(range(len(losses_lstm)),losses_lstm);

In [ ]:
lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test)):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

In [ ]:
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)

In [ ]:
error_LSTM = mean_squared_error(test, lstm_predictions) ** 0.5
print('Test RMSE: %.3f' % error_LSTM)

In [ ]:
RollBack = pd.concat([RollBack,pd.DataFrame({'LSTM':np.concatenate(lstm_predictions, axis=0)})],axis=1)
RollBack.head()

In [ ]:
RollBack[['TEST', 'LSTM']].plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=15);

In [ ]:
RollBack.plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=15);

# 5. Conclusiones

In [ ]:
RollBack = pd.concat([RollBack,pd.DataFrame({'Time':data.Log_Production.index[size:]})],axis=1)
RollBack.head()

In [ ]:
RollBack.set_index('Time', inplace=True)
RollBack.head()

In [ ]:
RollBack.plot(figsize=(20,5), linewidth=2, fontsize=10)
plt.xlabel('time', fontsize=15);

In [ ]:
Error = pd.DataFrame({"Models":["ARIMA", "SARIMA", "Prophet", "LSTM"],
                      "RMSE Log" : [error_ARIMA, error_SARIMA, error_PROPHET, error_LSTM]})
Error

In [ ]:
print('Test RMSE ARIMA: %.3f' % mean_squared_error(np.exp(RollBack.TEST), np.exp(RollBack.ARIMA)))
print('Test RMSE SARIMA: %.3f' % mean_squared_error(np.exp(RollBack.TEST), np.exp(RollBack.SARIMA)))
print('Test RMSE Prophet: %.3f' % mean_squared_error(np.exp(RollBack.TEST), np.exp(RollBack.Prophet)))
print('Test RMSE LSTM: %.3f' % mean_squared_error(np.exp(RollBack.TEST), np.exp(RollBack.LSTM)))

Al comparar los RMSE obtenidos en las diferentes estimaciones, se obtiene que el Modelo SARIMA arroja el menor RSME (84.416), seguido del Modelo Prophet (RSME = 133.123). Por otra parte, cuando se grafican la series estimadas versus la observada, se encuentra que la serie resultado de los modelos Prophet y SARIMA son los de mejo ajuste a la serie de análisis. Sin embargo, los residuales del modelo SARIMA no se distribuyen normal. Por lo cual recomendamos el uso del modelo Prophet que no requiere de este supuesto para la realización de pronósticos de la producción de cerveza.

En términos de negocio, debido a la misma naturaleza del mercado de Cervezas, es importante  tener una buena predicción especialmente para los periodos estacionales, ya que son los de mayor foco para lanzamientos de nuevas marcas o campañas específicas. Dicho esto, el modelo SARIMA funciona mejor pero tendríamos que hacer alguna transformación o corrección en los datos train para garantizar la normalidad en los residuales, debido a esto se recomienda utilizar el modelo Prophet.

En términos de Escalabilidad tanto el modelo Prophet como el SARIMA funcionan muy bien, ya que el algoritmo en su proceso iterativo evalúa múltiples modelos, eligiendo el de mejor performance. Podría ser muy interesante para los Fabricantes aplicar estos modelos en Datos de más países donde tienen participación, ya que en general la naturaleza de los datos se mantiene en los diferentes países.

En cuanto al procesamiento, consume menos hardware procesar el prophet que el modelo SARIMA, así mismo, al medir sus tiempos de ejecución el modelo SARIMA consume más tiempo de ejecución que el Prophet

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
np.exp(RollBack[['SARIMA','TEST']]).plot(figsize=(20,5), linewidth=2, fontsize=10, ax = ax[0])
np.exp(RollBack[['Prophet','TEST']]).plot(figsize=(20,5), linewidth=2, fontsize=10, ax = ax[1])
plt.xlabel('time', fontsize=15);
#plt.show()